<h3> Pojedynczy problem </h3>

In [ ]:
def single_stokes_stationary(wall, obstacles, debug_mode = False, dirichlet_z = [],
                                  dirichlet_f = [], scale_axes = 1, scale_elements_width = 1, scale_elements_height = 1, viscosity = 1, force_option = 1, returns = False):
    """
        Algorytm rozwiązujący przepływ paraboliczny. Wizualizuje rozwiązanie.
        input:  węzły wall i węzły przeszkód: lewy_dolny, prawy_dolny, prawy_górny, lewy_górny
                wall = np.array([w1,w2,w3,w4]), gdzie w_i = lista, [x_y,y_i];
                obstacles = np.array([[o_1_1,o_1_2,o_1_3,o_1_4],...,[o_q_1,o_q_2,o_q_3,o_q_4]]),
                gdzie o_q_j = [x_q_j,y_q_j], przeszkody podane wg. kolumn od lewej do prawej i wg. wierszy od dołu do góry;  
                debug_mode = True/False, czy obliczać dodatkowe obiekty;
                dirichlet_z = list(), lista z brzegiem Dirichleta gdzie u == 0;
                dirichlet_f = list(), lista z brzegiem Dirichleta gdzie u = g_D;
                scale_axes = float > 0, przeskalowanie obu osi;
                scale_elements_width = int > 0, przeskalowanie szerokości elementu siatki;
                scale_elements_height = int > 0, przeskalowanie długości elementu siatki;
                viscosity = float > 0, lepkość;
                force_option = int spośród {1,2,3}, wybór wektora sił funkcji force_vector();
                returns = True/False, czy zwrócić output;
        output: Tylko jeśli returns = True,
                nodes = np.array(), tablica węzłów siatki dla ciśnienia;
                P2nodes = np.array(), tablica węzłów siatki dla prędkości;
                ksi_x[:,1] = np.array(), wektor rozwiązania u_1;
                ksi_y[:,1] = np.array(), wektor rozwiązania u_2;
                gamma[:,1] = np.array(), wektor rozwiązania p;
                M_P1 = scipy.sparse.csr_matrix(), macierz masy dla P1;
                M_P2 = scipy.sparse.csr_matrix(), macierz masy dla P2;
                A = scipy.sparse.csr_matrix(), macierz sztywności dla P2;
                period = list(), czasy obliczeń poszczególnych etapów.
    """
    
    global period  # lista przechowująca czasy poszczególnych etapów 
    period = [0,0,0,0]
    t1 = time.time()  # początkowy pomiar czasu etapu
    
    elements_byrows = ((wall[1][0] - wall[0][0])) * scale_elements_width  # liczba elementów w poziomie (bez uwzględnienia przeszkód)
    elements_bycolumns = ((wall[2][1] - wall[0][1])) * scale_elements_height # liczba elementów w pionie (bez uwzględnienia przeszkód)

    # Generujemy siatkę dla ciśnienia
    
    if debug_mode == True:
        connections, nodes, elements, area_counters = meshgenerator(wall, obstacles, elements_byrows,
                                                      elements_bycolumns, True, scale_axes)
    else:
        connections, nodes = meshgenerator(wall, obstacles, elements_byrows, elements_bycolumns, False, scale_axes)
    
    wall = wall * scale_axes  # skalujemy kanał i przeszkody do faktycznych rozmiarów
    obstacles = obstacles * scale_axes
    
    global wall_bottom  # informacje o kanale
    global wall_top
    global wall_right
    global wall_left
    wall_bottom = wall[0][1]
    wall_top = wall[2][1]
    wall_right = wall[1][0]
    wall_left = wall[0][0]

    length = wall[1][0] - wall[0][0]  # obliczenie długości i wysokości kanału
    height = wall[2][1] - wall[0][1]
    
    element_x_length = length / elements_byrows  # obliczenie przeciwprostokątnych trójkąta
    element_y_length = height / elements_bycolumns
    
    K = element_x_length * element_y_length / 2  # obliczenie powierzchni trójkąta
    
    global h_size  # obliczenie globalnego rozmiaru siatki
    h_size = np.around(( element_x_length ** 2 + element_y_length ** 2 ) ** (1/2),4)
    
    x_p = np.linspace(wall_left, wall_right, num=int(np.around(length/element_x_length,0))+1)  # obliczenie możliwych współrzędnych węzłów dla ciśnienia
    y_p = np.linspace(wall_bottom, wall_top, num=int(np.around(height/element_y_length,0))+1)
    
    # Generujemy siatkę dla prędkości
    
    P2nodes, P2connections = ChangeP1toP2Mesh(nodes, connections)
    t2 = time.time()
    period[0] = t2-t1  # dodanie pomiaru do listy, czas: triangulacja, utworzenie siatki dla węzłów ciśnienia i utworzenie siatki dla węzłów prędkości
    
    # Wizualizacja węzłów dla ciśnienia
    
    nodes_visualization(nodes)
    
    # Wizualizacja węzłów prędkości
    
    nodes_visualization(P2nodes, P = 'u') 
    
    # Konstrukcje wektorów i  macierzy
    
    t1 = time.time()
    force = force_vector(P2nodes.shape[0],force_option)  # wektor sił

    ksi_0 = np.zeros(2 * P2nodes.shape[0])  # warunki początkowe wektora prędkości
    ksi_0[:] = 0 

    M_P1 = mass_matrix_calculate(nodes, connections, K)  # macierze masy
    M_P2 = P2_mass_matrix_uniform_mesh(P2nodes, P2connections)
    A = P2_stiffness_matrix_uniform_mesh(P2nodes, P2connections)  # macierz sztywności
     
    D_x = divergence_matrix_uniform_mesh(nodes, P2nodes, connections, P2connections, 'x')  # podmacierze dywergencji
    D_y = divergence_matrix_uniform_mesh(nodes, P2nodes, connections, P2connections, 'y')  

    if not np.any(force):  # jeśli wektor sił zerowy
        b = np.zeros([2 * P2nodes.shape[0], 1])  # wektor obciążeń
    else: 
        b_x = P2_load_vector_calculate(P2nodes, P2connections, force)  # wektor obciążeń
        b = np.concatenate((b_x,b_x), axis=0)  # wektor b jest dla u_y taki sam jak dla u_x
        
    x_u = np.linspace(wall_left, wall_right, num=int(np.around(2*length/element_x_length,0))+1)  # obliczenie możliwych współrzędnych węzłów dla prędkości
    y_u = np.linspace(wall_bottom, wall_top, num=int(np.around(2*height/element_y_length,0))+1)

    t2 = time.time()
    period[1] = t2-t1  # czas: konstrukcja układu równań cz.1

    # Rozpatrzenie warunków brzegowych
    
    C, dirichlet_zero_nodes, dirichlet_fun_nodes, no_boundary = set_boundaries(obstacles, x_u, y_u, P2nodes, dirichlet_z, dirichlet_f) 
    
    # Obliczenie rozwiązania
    
    ksi_x, ksi_y, gamma = solver_stokes_stationary(A, D_x, D_y, b, ksi_0, dirichlet_zero_nodes, dirichlet_fun_nodes, no_boundary, P2nodes) 

    t1 = time.time()
    
    # Wizualizacja normy euklidesowej prędkości
    
    ksi_norm = euclidean_norm(ksi_x,ksi_y)
    function_visualization(C, x_u, y_u, P2nodes, ksi_norm, 'u')

    # Wizualizacja pola wektorowego prędkości
    
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,9))
    plt.quiver(P2nodes[:,0], P2nodes[:,1], ksi_x[:,1], ksi_y[:,1], ksi_norm[:,1])
    plt.colorbar()
    plt.title('Wizualizacja pola wektorowego prędkości, h = ' + str(h_size) + ', \u03BD = ' + str(viscosity) )
    plt.show()
        
    # Wizualizacja ciśnienia
    
    C = np.zeros((len(y_p), len(x_p)))
    function_visualization(C, x_p, y_p, nodes, gamma, 'p')

    if obstacles.size == 0:  # jeśli to przepływ bez przeszkody, to porównujemy z rozwiązaniem dokładnym
        parabolic_flow_supremum_error(ksi_x[:,1], ksi_y[:,1], gamma[:,1], nodes, P2nodes)  # norma supremum błędu
        
    t2 = time.time()
    period[3] = t2-t1  # czas: preprocessing
    
    if returns:
        return nodes, P2nodes, ksi_x[:,1], ksi_y[:,1], gamma[:,1], M_P1, M_P2, A, period
    

<h3> Problem dla wielu siatek </h3>

In [1]:
def multiple_stokes_stationary(wall, obstacles, debug_mode = False, dirichlet_z = [],
                                  dirichlet_f = [], scale_axes = 1, scale_elements_width = [2,4,8], scale_elements_height = [2,4,8], viscosity=1, force_option=1):
    """
        Algorytm rozwiązujący przepływ paraboliczny dla wielu siatek. Wizualizuje rozwiązanie, jego błąd i czas obliczeń dla każdej z siatek.
        input:  węzły wall i węzły przeszkód: lewy_dolny, prawy_dolny, prawy_górny, lewy_górny
                wall = np.array([w1,w2,w3,w4]), gdzie w_i = lista, [x_y,y_i];
                obstacles = np.array([[o_1_1,o_1_2,o_1_3,o_1_4],...,[o_q_1,o_q_2,o_q_3,o_q_4]]),
                gdzie o_q_j = [x_q_j,y_q_j], przeszkody podane wg. kolumn od lewej do prawej i wg. wierszy od dołu do góry;
                debug_mode = True/False, czy obliczać dodatkowe obiekty;
                dirichlet_z = list(), lista z brzegiem Dirichleta gdzie u == 0;
                dirichlet_f = list(), lista z brzegiem Dirichleta gdzie u = g_D;
                scale_axes = float > 0, przeskalowanie obu osi;
                scale_elements_width = list(int>0), lista przeskalowań szerokości elementu dla kolejnych siatek, liczby rosnąco
                scale_elements_height = list(int>0), lista przeskalowań długości elementu dla kolejnych siatek, liczby rosnąco
                viscosity = float > 0, lepkość;
                force_option = int spośród {1,2,3}, wybór wektora sił funkcji force_vector();
    """

    hm_problems = len(scale_elements_width)  # ilość rozpatrywanych siatek
    equations_size = []  # wielkość układu równań
    
    arr_nodes = []  # węzły ciśnienia
    arr_P2nodes = []  # węzły prędkości
    arr_ksi_x = []  # rozwiązania u_1
    arr_ksi_y = []  # rozwiązania u_2
    arr_gamma = []  # rozwiązania p
    arr_P1_M = []  # macierze masy dla P1
    arr_P2_M = []  # macierze masy dla P2
    arr_P2_A = []  # macierze sztywności dla P2
    arr_period = []  # czasy etapów
    h = []  # globalna wielkość siatki

    for i in range(hm_problems):  # alokujemy miejsca dla outputów
        arr_nodes.append(0)
        arr_P2nodes.append(0)
        arr_ksi_x.append(0)
        arr_ksi_y.append(0)
        arr_gamma.append(0)
        arr_P1_M.append(0)
        arr_P2_M.append(0)
        arr_P2_A.append(0)
        arr_period.append(0)
        h.append(0)

    # Obliczamy rozwiązania dla kolejnych siatek
        
    for i in range(hm_problems):

        arr_nodes[i], arr_P2nodes[i], arr_ksi_x[i], arr_ksi_y[i], arr_gamma[i], arr_P1_M[i], arr_P2_M[i], arr_P2_A[i], arr_period[i] = single_stokes_stationary(wall=mywall, obstacles=myobstacles, debug_mode = debug_mode,
                                  dirichlet_z = dirichlet_zero, dirichlet_f = dirichlet_fun, scale_axes = scale_axes, scale_elements_width = scale_elements_width[i], scale_elements_height = scale_elements_height[i], returns = True, viscosity=viscosity)
        h[i] = ( arr_nodes[i][1,0] - arr_nodes[i][0,0] ) * (2 ** (1/2))  # liczymy globalną wielkość siatki
        equations_size.append(len(arr_nodes[i]) + 2 * len(arr_P2nodes[i]))  # obliczamy wielkość układu równań
        
    # Obliczamy błędy    
        
    L2_errors, H1_errors = calculate_errors(arr_nodes, arr_P2nodes, arr_ksi_x, arr_ksi_y, arr_gamma, arr_P1_M, arr_P2_M, arr_P2_A)

    # Wizualizujemy błędy
    
    
    a = H1_errors[0]/h[0]**2
    b = L2_errors[0]/h[0]
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
    ax.loglog(1/np.array(h[:-1]), L2_errors, 'o--g', label='Błąd ciśnienia w normie L2')
    ax.loglog(1/np.array(h[:-1]), H1_errors, 'o--r', label='Błąd wektora prędkości w normie H1')
    ax.loglog(1/np.array(h[:-1]), np.array(h[:-1])*b, color='y', label='C*h')
    ax.loglog(1/np.array(h[:-1]), ((np.array(h[:-1])))**2*a, color='b', label='V*h^2')
    ax.set_title('Porównanie błędów w skali podwójnie logarytmicznej')
    ax.set_xlabel('1/h')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
              fancybox=True, shadow=True, ncol=5)
    plt.show()

    # Wizualizujemy czas obliczeń
    
    arr_period = np.array(arr_period)
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
    ax.loglog(equations_size, arr_period[:,0], 'o--', label='Triangulacja, utworzenie siatek')
    ax.loglog(equations_size, arr_period[:,1], 'o--', label='Konstrukcja układu równań')
    ax.loglog(equations_size, arr_period[:,2], 'o--', label='Obliczenie rozwiązania')
    ax.loglog(equations_size, arr_period[:,3], 'o--', label='Postprocessing')
    ax.set_title('Porównanie czasu obliczeń poszczególnych etapów w skali podwójnie logarytmicznej')
    ax.set_xlabel('Liczba równań')
    ax.set_ylabel('Czas (s)')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
              fancybox=True, shadow=True, ncol=2)
    plt.show()
    
    print('arr_period0: ', arr_period[:,0])
    print('arr_period1: ', arr_period[:,1])
    print('arr_period2: ', arr_period[:,2])
    print('arr_period3: ', arr_period[:,3])
    
    print('h: ',h)
    print('L2_errors: ',L2_errors)
    print('H1_errors: ',H1_errors)
    
